In [35]:
#  Answer the questions sequentially with proper question number in comment at the top.
#  Each question carries different marks.      
# All the necessary imports
from pyspark import SparkContext 
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import Row
sqlContext = SQLContext(sc)
sc.stop()

In [1]:
#01 We have fifa world cup datasets on which we want certain analysis.
#   a) Do some data cleaning on column name Attendance as to remove '.' character ?(eg. '590.554' => 590554 in integer)    2
#   b) List the countries based on the number of world Cup final they have won in descending order?(Note: required columns:WinningCountry,#fifaWCwinner)     2
#   c) List the countries based on total number of audience watched their final world cup match  and order them in descending order?(Note: required columns:Country,TotalAttendance)  3
#   d) List out the countries who came into fifa final but never won and also winner countries who never came in second position?     3

df = (sqlContext.read.format('com.databricks.spark.csv')
      .options(header='true', inferschema='true')
      .load('/home/affine/Desktop/Affine/07PySpark/Questions_Data_Pyspark_Test/FIFA/WorldCups.csv')
      )
df.printSchema()
df.show()

root
 |-- Year: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Winner: string (nullable = true)
 |-- Runners-Up: string (nullable = true)
 |-- Third: string (nullable = true)
 |-- Fourth: string (nullable = true)
 |-- GoalsScored: integer (nullable = true)
 |-- QualifiedTeams: integer (nullable = true)
 |-- MatchesPlayed: integer (nullable = true)
 |-- Attendance: string (nullable = true)

+----+------------+----------+--------------+-----------+--------------+-----------+--------------+-------------+----------+
|Year|     Country|    Winner|    Runners-Up|      Third|        Fourth|GoalsScored|QualifiedTeams|MatchesPlayed|Attendance|
+----+------------+----------+--------------+-----------+--------------+-----------+--------------+-------------+----------+
|1930|     Uruguay|   Uruguay|     Argentina|        USA|    Yugoslavia|         70|            13|           18|   590.549|
|1934|       Italy|     Italy|Czechoslovakia|    Germany|       Austria|         70|

In [3]:
#Solution 01 a):
df = df.withColumn("Attendance", regexp_replace('Attendance', '[.]', '').cast(IntegerType()))

#Checks:
df.printSchema() #Datatype would change to Integer,double or float but not string
df.select('Attendance').show() #The '.' character  should be removed from Attendance column 

root
 |-- Year: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Winner: string (nullable = true)
 |-- Runners-Up: string (nullable = true)
 |-- Third: string (nullable = true)
 |-- Fourth: string (nullable = true)
 |-- GoalsScored: integer (nullable = true)
 |-- QualifiedTeams: integer (nullable = true)
 |-- MatchesPlayed: integer (nullable = true)
 |-- Attendance: integer (nullable = true)

+----------+
|Attendance|
+----------+
|    590549|
|    363000|
|    375700|
|   1045246|
|    768607|
|    819810|
|    893172|
|   1563135|
|   1603975|
|   1865753|
|   1545791|
|   2109723|
|   2394031|
|   2516215|
|   3587538|
|   2785100|
|   2705197|
|   3359439|
|   3178856|
|   3386810|
+----------+



In [4]:
#Solution 01 b):
df1=df.groupBy('Winner').count().sort('count', ascending=False).select(col('Winner').alias('WinningCountry'),col('count').alias("#fifaWCwinner"))
df1.show()
#Results should match

+--------------+-------------+
|WinningCountry|#fifaWCwinner|
+--------------+-------------+
|        Brazil|            5|
|         Italy|            4|
|    Germany FR|            3|
|     Argentina|            2|
|       Uruguay|            2|
|        France|            1|
|       England|            1|
|         Spain|            1|
|       Germany|            1|
+--------------+-------------+



In [5]:
#Solution 01 c):
df1=df.select('Winner','Attendance')
df2=df.select('Runners-Up','Attendance')
df3=df1.union(df2)
df3.groupBy('Winner').sum('Attendance').sort('sum(Attendance)', ascending=False).select(col('Winner').alias('Country'),col('sum(Attendance)').alias("TotalAttendance")).show()

#Results should match

+--------------+---------------+
|       Country|TotalAttendance|
+--------------+---------------+
|        Brazil|       13440038|
|         Italy|       11399375|
|    Germany FR|       11217464|
|     Argentina|       10433396|
|   Netherlands|        6590400|
|        France|        6144539|
|       Germany|        6092007|
|         Spain|        3178856|
|       Uruguay|        1635795|
|       England|        1563135|
|Czechoslovakia|        1256172|
|       Hungary|        1144307|
|        Sweden|         819810|
+--------------+---------------+



In [6]:
#Solution 01 d)
df.select('Winner').subtract(df.select('Runners-up')).show()
df.select('Runners-up').subtract(df.select('Winner')).show()

+-------+
| Winner|
+-------+
|  Spain|
|Uruguay|
|England|
+-------+

+--------------+
|    Runners-up|
+--------------+
|        Sweden|
|Czechoslovakia|
|       Hungary|
|   Netherlands|
+--------------+



In [7]:
# 2)   Solve below parts using rdd only.
#      a) Create a rdd from DJBravolyrics.txt File.     1                                             
#      b) Calculate Total number of  words in song.     2
#      c) Order each distinct line in text file based on their length and see pattern for next question.   2
#      d) Find out all the famous personaties in lyric.   3



In [8]:
#Solution 02 a)
rdd=sc.textFile("/home/affine/Desktop/Affine/07PySpark/Questions_Data_Pyspark_Test/DJBravolyrics.txt")
#rdd.collect()

In [9]:
#Solution 02 b)
rdd2=rdd.map(lambda k: len(k.split(' ')))
rdd2.sum()

259

In [10]:
#Solution 02 c)
rdd3=rdd.distinct().sortBy(len)
rdd3.collect()

["Don't forget",
 'Obama a champion',
 'Wendy a champion',
 'DJ Bravo DJ Bravo',
 'Champion champion',
 'Beenie a champion',
 'Serena a champion',
 'Bounty a champion',
 'Mandela a champion',
 'Lara is a champion',
 'Champion, champion',
 'Bunji is a champion',
 'Gayle is a champion',
 '958 BOLT a champion',
 'Machel is a champion',
 'Pollard is a champion',
 'Shelly-Ann a champion',
 'Michel Jordan a champion',
 'We love soca and dancehall',
 'Every vincy and every banjan',
 'We love cricket and football',
 'Champion we are real champion',
 'Every trini and every yard man',
 "Don't vex if your name not call",
 'Everybody know say Bravo a champion',
 'From ah dancing ah my name your call',
 'We have to leave time to bat some ball',
 'Everybody know Chris Gayle is a champion']

In [11]:
#Solution 02 d)
rdd3.filter(lambda x:'champion' in x).map(lambda x: x.replace("is a champion","").replace("a champion","").replace("Everybody know ","")).sortBy(len).collect()[:-3]


['Lara ',
 'Obama ',
 'Wendy ',
 'Bunji ',
 'Gayle ',
 'Beenie ',
 'Serena ',
 'Bounty ',
 'Machel ',
 'Mandela ',
 'Pollard ',
 '958 BOLT ',
 'say Bravo ',
 'Shelly-Ann ',
 'Chris Gayle ',
 'Michel Jordan ']

In [2]:
# 3) a) Create a Dataframe from cars.txt file and also change columns' dataype.
#       {Accelerate:double/float,Cylinders:long/int,Engine_Disp/int:long,Horsepower:long/int,MPG:long/int,
#        Model:string,Origin:string,Weight:long/int,Year:long/int,make:string}     3   
#    b) Add one more column to your dataframe having mileage in kilometers per liter  Note(1MPG=0.43 KMPL)   2
#    c) Count the number of cars which are having engine displacement more than 200cc and list then in ascending oder.      2                
#    d) Find average weight of cars by their origin and year.               2                                



In [8]:
#3 a)
rdd = sc.textFile("/home/affine/Desktop/Affine/07PySpark/Questions_Data_Pyspark_Test/cars.txt")
rdd1=rdd.map(lambda x:x.split("\t"))
header=rdd1.first()
rdd_data=rdd1.filter(lambda line:line!=header)
rdd_data1=rdd_data.map(lambda x:Row(make=x[0],Model=x[1],MPG=int(x[2]),Cylynders=int(x[3]),Engine_Disp=int(x[4]),Horsepower=int(x[5]),Weight=int(x[6]),Accelerate=float(x[7]),Year=int(x[8]),Origin=x[9]))
df=sqlContext.createDataFrame(rdd_data1)

In [9]:
df.printSchema()

root
 |-- Accelerate: double (nullable = true)
 |-- Cylynders: long (nullable = true)
 |-- Engine_Disp: long (nullable = true)
 |-- Horsepower: long (nullable = true)
 |-- MPG: long (nullable = true)
 |-- Model: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Weight: long (nullable = true)
 |-- Year: long (nullable = true)
 |-- make: string (nullable = true)



In [21]:
#b
def mpg_to_kpl(x):
    return float(x)*0.43
mpg_to_kpl_udf= udf(mpg_to_kpl, FloatType())

df2= df.withColumn("KMPL",mpg_to_kpl_udf(df["MPG"]))
df2.show()

+----------+---------+-----------+----------+---+--------------------+--------+------+----+---------+----+
|Accelerate|Cylynders|Engine_Disp|Horsepower|MPG|               Model|  Origin|Weight|Year|     make|KMPL|
+----------+---------+-----------+----------+---+--------------------+--------+------+----+---------+----+
|       8.5|        8|        390|       190| 15|  amc ambassador dpl|American|  3850|  70|      amc|6.45|
|      15.0|        6|        199|        90| 21|         amc gremlin|American|  2648|  70|      amc|9.03|
|      15.5|        6|        199|        97| 18|          amc hornet|American|  2774|  70|      amc|7.74|
|      12.0|        8|        304|       150| 16|       amc rebel sst|American|  3433|  70|      amc|6.88|
|      10.0|        8|        455|       225| 14|buick estate wago...|American|  3086|  70|    buick|6.02|
|      11.5|        8|        350|       165| 15|   buick skylark 320|American|  3693|  70|    buick|6.45|
|      12.0|        8|        307|   

In [28]:
#c)
df.filter(df["Engine_Disp"]>200).orderBy("Engine_Disp").show()


+----------+---------+-----------+----------+---+--------------------+--------+------+----+---------+
|Accelerate|Cylynders|Engine_Disp|Horsepower|MPG|               Model|  Origin|Weight|Year|     make|
+----------+---------+-----------+----------+---+--------------------+--------+------+----+---------+
|      15.5|        6|        225|       105| 16|plymouth satellit...|American|  3439|  71| plymouth|
|      15.5|        6|        232|       100| 18|         amc matador|American|  3288|  71|      amc|
|      13.0|        6|        232|       100| 19|         amc gremlin|American|  2634|  71|      amc|
|      14.5|        6|        250|        88| 18|        ford mustang|American|  3139|  71|     ford|
|      15.5|        6|        250|       100| 17|chevrolet chevell...|American|  3329|  71|chevrolet|
|      15.0|        6|        250|       100| 19|    pontiac firebird|American|  3282|  71|  pontiac|
|      15.5|        6|        250|        88| 19|     ford torino 500|American|  3

In [24]:
#d
df.groupBy("Origin","Year").agg(avg("Weight")).show()

+--------+----+------------------+
|  Origin|Year|       avg(Weight)|
+--------+----+------------------+
|American|  70|            3716.5|
|Japanese|  70|            2251.0|
|European|  70|            2309.2|
|American|  71|3472.9473684210525|
|European|  71|            2024.0|
|American|  72|3807.6666666666665|
|Japanese|  71|            1936.0|
+--------+----+------------------+



In [34]:
#04)In Retail excel file see the data dictionary and understand the relationship among different tables.
#   a)Create a column called Profit in Products table.(Price-Cost)   1
#   b)Find out the top 3 high profit customer.                           3
#   c)What are the total sales by Month/Quarter/Year.                    3

directory="/home/affine/Desktop/Affine/07PySpark/Questions_Data_Pyspark_Test/retail/"

Orders=sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load(directory+'Orders.csv')
Employees=sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load(directory+'Employees.csv')
Customers=sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load(directory+'Customers.csv')
Orders=sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load(directory+'Orders.csv')
OrderDetails=sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load(directory+'OrderDetails.csv')
Products=sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load(directory+'Products.csv')
Shippers=sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load(directory+'Shippers.csv')
Suppliers=sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load(directory+'Suppliers.csv')


root
 |-- SupplierID: integer (nullable = true)
 |-- SupplierName: string (nullable = true)
 |-- ContactName: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- PostalCode: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Phone: string (nullable = true)



Row(SupplierID=1, SupplierName='Exotic Liquid', ContactName='Charlotte Cooper', Address='49 Gilbert St.', City='London', PostalCode='EC1 4SD', Country='UK', Phone='(171) 555-2222')

In [ ]:
# 5) a) Assume you have a data which is 200MB in size and a cluster of 8 machines such that each machine 
#       comprises of 64GB Ram.Assume your data is partitioned into 200 small files.
#       Which is the best option to choose in terms of time complexity and Why ??
#         i)  Reduce the number of partitions of data and bring it to single machine.
#         ii) Increase the number of partitions of data.                                               2 Marks 

#    b) Suppose you have a data divided into 200 small partitions. You want it to increase to 500 partitions.
#       Which method would you use for it and why do you use that method ??                            2 Marks

#    c) Suppose you have a data divided into 200 small partition . You want it to decrease to 50 partitions.
#       Which method would you use for it and Why do you use that method.          2Marks